<a href="https://colab.research.google.com/github/ntomasini/MaxiDrift/blob/main/MaxicircleDriftModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Simulation of maxicircle drift
# FINAL CODE: Simulation over multiple maxicircle values
import numpy as np
import pandas as pd

def simulate_without_bottlenecks(initial_kinetoplasts, max_population, generations, maxicircles_per_parasite):
    """
    Simulates parasite population dynamics without bottlenecks, over a specified number of generations.
    Each parasite contains 'maxicircles_per_parasite' maxicircles in its kinetoplast.

    Process details:
    1. Each generation, the entire set of kinetoplasts is duplicated (doubling the maxicircles).
    2. The duplicated maxicircles are shuffled randomly.
    3. The first half of the shuffled array is assigned to a new kinetoplast (new parasite),
       and the second half is assigned to another new kinetoplast (another parasite).
    4. If the number of new kinetoplasts exceeds 'max_population', it is randomly trimmed down to that limit.
    5. For each kinetoplast, we count how many 'A' vs. 'B' maxicircles it carries.

    Parameters:
    ----------
    initial_kinetoplasts : list of lists
        Each element is a list representing the initial maxicircles of one parasite.
        Example: [['A']*10 + ['B']*10] means 20 maxicircles in total, half 'A' and half 'B'.
    max_population : int
        Maximum allowed number of parasites in the population.
    generations : int
        Number of generations to simulate.
    maxicircles_per_parasite : int
        Number of maxicircles each parasite has (constant across all parasites).

    Returns:
    --------
    pd.DataFrame
        A DataFrame with one row per kinetoplast per generation, containing:
        - Generation (int)
        - Kinetoplast (str) - an identifier
        - A (int) - number of 'A' maxicircles
        - B (int) - number of 'B' maxicircles
    """
    current_kinetoplasts = np.array(initial_kinetoplasts, dtype='U1')
    results = []
    half = maxicircles_per_parasite

    for gen in range(generations):
        next_generation_list = []
        for kinetoplast in current_kinetoplasts:
            # Duplicate the array of maxicircles
            duplicated = np.concatenate([kinetoplast, kinetoplast])
            # Shuffle randomly to simulate random segregation
            np.random.shuffle(duplicated)
            # Split into two new kinetoplasts
            new_kin_1 = duplicated[:half]
            new_kin_2 = duplicated[half:(2 * half)]
            next_generation_list.append(new_kin_1)
            next_generation_list.append(new_kin_2)

        # Convert the list of arrays into a single NumPy array
        next_generation = np.array(next_generation_list, dtype='U1')

        # If population exceeds max_population, randomly sample to keep the limit
        if len(next_generation) > max_population:
            indices = np.random.choice(len(next_generation), max_population, replace=False)
            next_generation = next_generation[indices]

        # Count how many 'A' and 'B' each kinetoplast has
        A_mask = (next_generation == 'A')
        A_counts = np.sum(A_mask, axis=1)
        B_counts = half - A_counts

        # Store intermediate results for each kinetoplast
        for idx in range(len(next_generation)):
            results.append({
                "Generation": gen + 1,
                "Kinetoplast": f"Kinetoplast {idx + 1}",
                "A": A_counts[idx],
                "B": B_counts[idx]
            })

        # Move on to the next generation
        current_kinetoplasts = next_generation

    return pd.DataFrame(results)


def calculate_fixation_proportions(simulation_results):
    """
    Calculates fixation proportions (A fixed, B fixed) and the proportion
    of kinetoplasts that are not fixed (i.e., they still have both 'A' and 'B').

    It groups the data by 'Generation' and then computes:
    - Proportion of kinetoplasts with 0 'B'  => "Proportion Fixed A"
    - Proportion of kinetoplasts with 0 'A'  => "Proportion Fixed B"
    - Proportion of kinetoplasts with both 'A' and 'B' => "Proportion Not Fixed"

    Parameters:
    -----------
    simulation_results : pd.DataFrame
        DataFrame returned by simulate_without_bottlenecks, which has columns:
        ['Generation', 'Kinetoplast', 'A', 'B'].

    Returns:
    --------
    pd.DataFrame
        One row per generation, with columns:
        ['Generation', 'Proportion Fixed A', 'Proportion Fixed B', 'Proportion Not Fixed'].
    """
    grouped = simulation_results.groupby("Generation")
    rows = []
    for generation_value, group_df in grouped:
        propA = (group_df["B"] == 0).mean()
        propB = (group_df["A"] == 0).mean()
        propNot = ((group_df["A"] > 0) & (group_df["B"] > 0)).mean()

        rows.append({
            "Generation": generation_value,
            "Proportion Fixed A": propA,
            "Proportion Fixed B": propB,
            "Proportion Not Fixed": propNot
        })
    return pd.DataFrame(rows)


def run_multiple_replicas_no_bottlenecks(num_replicas, initial_kinetoplasts, max_population, generations, maxicircles_per_parasite):
    """
    Executes multiple (num_replicas) runs of the simulation (simulate_without_bottlenecks),
    and then concatenates the fixation proportions for each generation, labeling them by replica.

    Parameters:
    -----------
    num_replicas : int
        How many independent simulation runs to perform.
    initial_kinetoplasts : list of lists
        Initial configuration of kinetoplasts (A/B distribution).
    max_population : int
        Maximum population size to allow in the simulation.
    generations : int
        Number of generations to simulate for each run.
    maxicircles_per_parasite : int
        Number of maxicircles per parasite (fixed for all individuals).

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the generation-by-generation fixation proportions
        for each of the replicas, with an additional column "Replica" labeling
        each replicate run.
    """
    fixation_proportions_all = []
    for replica in range(1, num_replicas + 1):
        simulation_results = simulate_without_bottlenecks(
            initial_kinetoplasts=initial_kinetoplasts,
            max_population=max_population,
            generations=generations,
            maxicircles_per_parasite=maxicircles_per_parasite
        )
        fixation_proportions = calculate_fixation_proportions(simulation_results)
        fixation_proportions["Replica"] = replica
        fixation_proportions_all.append(fixation_proportions)

    return pd.concat(fixation_proportions_all, ignore_index=True)


# General parameters
max_population = 10000 #@param {"type":"integer"}
generations = 150 #@param {"type":"integer"}
num_replicas = 10 #@param {"type":"integer"}

# Maxicircle values to evaluate: 4, 12, 20, ..., 100
starting_maxicircles = 8 #@param {"type":"integer"}
increment = 8 #@param {"type":"integer"}
max_maxicircles = 100 #@param {"type":"integer"}
select_bias = 87.5 # @param ["50","75","87.5"] {"type":"raw"}
#@markdown bias 50= the hybrid received the same number of maxicircles from each parent, 75= the hybrid received 75% of the maxicircles from one parent, 87.5=the hybrid received 87.5% of the maxicircles from one parent. Each number of tested maxicircles should be a multiple of 2, 4 or 8 depending on the selected bias.
#maxicircle distribution


maxicircles_values = range(starting_maxicircles, max_maxicircles + 1, increment)

# List to compile the "Not Fixed" proportion (mean and std) across all simulations
compiled_not_fixed = []


for maxi in maxicircles_values:
    # Define half A and half B so that total is 'maxi'
    # (assuming 'maxi' is an even number)
    half_A = int(maxi * (select_bias/100))
    half_B = int(maxi * (1 - select_bias/100))

    initial_kinetoplasts = [['A'] * half_A + ['B'] * half_B]

    # Run the simulation for this maxicircle number
    fixation_proportions = run_multiple_replicas_no_bottlenecks(
        num_replicas=num_replicas,
        initial_kinetoplasts=initial_kinetoplasts,
        max_population=max_population,
        generations=generations,
        maxicircles_per_parasite=maxi
    )

    # Create a file name containing the parameters for the detailed output
    file_name = f"fixation_proportions_replicas_{num_replicas}reps_{generations}gens_{max_population}pop_{maxi}maxi.csv"
    fixation_proportions.to_csv(file_name, index=False)

    # Compute mean and std by generation
    gen_stats = (
        fixation_proportions
        .groupby("Generation")[["Proportion Fixed A", "Proportion Fixed B", "Proportion Not Fixed"]]
        .agg(['mean', 'std'])
        .reset_index()
    )

    # Rename columns for clarity (flatten multi-index)
    gen_stats.columns = [' '.join(col).strip() for col in gen_stats.columns.values]

    # Save the summary (mean/std) to a separate file
    file_name_gen = f"mean_std_fixation_proportions_per_generation_{num_replicas}reps_{generations}gens_{max_population}pop_{maxi}maxi.csv"
    gen_stats.to_csv(file_name_gen, index=False)

    # Gather "Proportion Not Fixed" mean and std for each generation
    # and append to a global list for final compilation
    for _, row in gen_stats.iterrows():
        compiled_not_fixed.append({
            "maxicircles_per_parasite": maxi,
            "Generation": int(row["Generation"]),
            "Proportion Not Fixed mean": row["Proportion Not Fixed mean"],
            "Proportion Not Fixed std": row["Proportion Not Fixed std"]
        })

# Convert the compiled data to a DataFrame
compiled_df = pd.DataFrame(compiled_not_fixed)

# Save a single CSV with "Not Fixed" proportions for all maxicircle values
compiled_file_name = "compiled_not_fixed_proportions_all_maxi.csv"
compiled_df.to_csv(compiled_file_name, index=False)

print("Simulations completed!")
print(f"Individual files have been generated for each maxicircle value (4 to 100 in steps of 8).")
print(f"A compiled file with 'Proportion Not Fixed' was also generated: {compiled_file_name}")


In [ ]:
# @title Graphs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the compiled file
compiled_df = pd.read_csv("compiled_not_fixed_proportions_all_maxi.csv")

# ----------------------------------------------------------------------------
#   STEP 1: Compute the "Proportion of Fixation" = 1 - "Proportion Not Fixed"
#           and store it in new columns.
# ----------------------------------------------------------------------------
compiled_df["Proportion Fixation mean"] = 1 - compiled_df["Proportion Not Fixed mean"]
compiled_df["Proportion Fixation std"] = compiled_df["Proportion Not Fixed std"]

# ----------------------------------------------------------------------------
#   STEP 2: Select series to plot
# ----------------------------------------------------------------------------
# Obtener valores únicos de maxicircles_per_parasite
unique_maxicircles = compiled_df["maxicircles_per_parasite"].unique()
print(f"Available series: {unique_maxicircles}")

# Solicitar al usuario seleccionar qué series graficar
selected_maxicircles = input("Enter the values of maxicircles_per_parasite to plot, separated by commas: ")
selected_maxicircles = [int(x.strip()) for x in selected_maxicircles.split(",")]

# Filtrar el DataFrame según la selección
filtered_df = compiled_df[compiled_df["maxicircles_per_parasite"].isin(selected_maxicircles)]

# ----------------------------------------------------------------------------
#   STEP 3: Plot "Proportion of Fixation" vs. Generation with error bars
# ----------------------------------------------------------------------------
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_df,
    x="Generation",
    y="Proportion Fixation mean",
    hue="maxicircles_per_parasite",
    marker='o',
    palette="tab10",
    errorbar=None
)

# Add error bars manually
for maxi in selected_maxicircles:
    subset = filtered_df[filtered_df["maxicircles_per_parasite"] == maxi]
    plt.errorbar(
        x=subset["Generation"],
        y=subset["Proportion Fixation mean"],
        yerr=subset["Proportion Fixation std"],
        fmt='none',
        ecolor='gray',
        alpha=0.5
    )

plt.title("Proportion of Fixation vs. Generation (by maxicircle number)")
plt.xlabel("Generation")
plt.ylabel("Fixation Probability (mean ± std)")

# Ajustar la leyenda fuera del gráfico
plt.legend(title="maxicircles_per_parasite", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Guardar la imagen
plt.savefig("Proportion_Fixation_vs_Generation.png", dpi=600, bbox_inches="tight")
plt.show()

# ----------------------------------------------------------------------------
#   STEP 4: Compute and Plot the probability of 6 consecutive fixations
# ----------------------------------------------------------------------------
compiled_df["Fixation_6_mean"] = compiled_df["Proportion Fixation mean"] ** 6
compiled_df["Fixation_6_std"] = (
    6 * (compiled_df["Proportion Fixation mean"] ** 5) * compiled_df["Proportion Fixation std"]
)

# Filtrar nuevamente el DataFrame
filtered_df = compiled_df[compiled_df["maxicircles_per_parasite"].isin(selected_maxicircles)]

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_df,
    x="Generation",
    y="Fixation_6_mean",
    hue="maxicircles_per_parasite",
    marker='o',
    palette="tab10",
    errorbar=None
)

# Add error bars manually for the binomial 6/6 probability
for maxi in selected_maxicircles:
    subset = filtered_df[filtered_df["maxicircles_per_parasite"] == maxi]
    plt.errorbar(
        x=subset["Generation"],
        y=subset["Fixation_6_mean"],
        yerr=subset["Fixation_6_std"],
        fmt='none',
        ecolor='gray',
        alpha=0.5
    )

plt.title("Probability of 6 Consecutive Fixations (p^6) vs. Generation")
plt.xlabel("Generation")
plt.ylabel("Probability(6/6 Fixation) (mean ± std)")

# Ajustar la leyenda fuera del gráfico
plt.legend(title="maxicircles per parasite", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Guardar la imagen
plt.savefig("Fixation_6_vs_Generation.png", dpi=600, bbox_inches="tight")
plt.show()
